In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
import geopandas as gpd
import numpy as np
from shapely.geometry import box, Polygon

def create_grid(geojson_path, output_path, city, resolution=500):
    # Load the GeoJSON file
    city_gdf = gpd.read_file(geojson_path)
    city_gdf = city_gdf.to_crs(epsg=3857)
    city_boundary = city_gdf.unary_union  # Combine all geometries into one

    # Get the bounding box of the city's boundary
    minx, miny, maxx, maxy = city_boundary.bounds

    # Create a grid of boxes
    grid_cells = []
    x_coords = np.arange(minx, maxx, resolution)
    y_coords = np.arange(miny, maxy, resolution)
    
    for x in x_coords:
        for y in y_coords:
            grid_cell = box(x, y, x + resolution, y + resolution)
            if grid_cell.intersects(city_boundary):
                grid_cells.append(grid_cell)

    # Create a GeoDataFrame for the grid
    grid_gdf = gpd.GeoDataFrame({'geometry': grid_cells}, crs=city_gdf.crs)
    # adding index to the grid using row number and city name
    grid_gdf['index'] = grid_gdf.index.values
    grid_gdf['index'] = city + '_' + grid_gdf['index'].astype(str)
    
    # Clip the grid to the city boundary
    clipped_grid = gpd.clip(grid_gdf, city_boundary)

    # Save the result as a GeoJSON file
    grid_gdf.to_file(output_path, driver="GeoJSON")
    
    return clipped_grid, grid_gdf, city_gdf

import os
files = os.listdir("Urban Local Boundary")

for file in files:
    city = file.split(".")[0]
    geojson_file = "Urban Local Boundary\\" + file
    output_file = "gridded\\" + file
    clipped_grid, grid_gdf, city_gdf = create_grid(geojson_file, output_file, city)



In [10]:
m = None
for file in os.listdir("gridded"):
    gdf = gpd.read_file("gridded\\" + file)
    if m is None:
        m = gdf.explore()
    else:
        gdf.explore(m=m)
